In [1]:
import sys, os, importlib
sys.path.insert(0, os.path.abspath(".."))

import src.qkernel_utils as qku
importlib.reload(qku)


<module 'src.qkernel_utils' from 'C:\\Users\\fourt\\Documents\\qkernel-benchmark\\src\\qkernel_utils.py'>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Two moons dataset
X, y = make_moons(n_samples=200, noise=0.2, random_state=42)

# Train/test split
from sklearn.model_selection import train_test_split
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.3, stratify=y, random_state=0)

# Standardize features
sc = StandardScaler().fit(Xtr)
Xtr_std, Xte_std = sc.transform(Xtr), sc.transform(Xte)

# Map to angles
Xtr_ang = qku.to_angles(Xtr_std)
Xte_ang = qku.to_angles(Xte_std)


In [3]:
dev = qku.make_device(wires=2)

k_elem_simple = qku.make_cached_kernel_element(dev, qku.feature_map)
k_elem_deep   = qku.make_cached_kernel_element(dev, qku.feature_map_deep)


In [4]:
K_full_simple = qku.quantum_kernel_matrix(Xtr_ang, Xtr_ang, k_elem_simple)
K_full_deep   = qku.quantum_kernel_matrix(Xtr_ang, Xtr_ang, k_elem_deep)

K_test_simple = qku.quantum_kernel_matrix(Xte_ang, Xtr_ang, k_elem_simple)
K_test_deep   = qku.quantum_kernel_matrix(Xte_ang, Xtr_ang, k_elem_deep)


In [5]:
def tune_qsvm_precomputed(K_full, K_test, ytr, yte, C_values):
    best_acc, best_C = -1, None
    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

    for C in C_values:
        scores = []
        for tr_idx, va_idx in skf.split(K_full, ytr):
            K_tr = K_full[np.ix_(tr_idx, tr_idx)]
            K_va = K_full[np.ix_(va_idx, tr_idx)]
            clf = SVC(kernel="precomputed", C=C).fit(K_tr, ytr[tr_idx])
            scores.append(accuracy_score(ytr[va_idx], clf.predict(K_va)))
        avg = np.mean(scores)
        if avg > best_acc:
            best_acc, best_C = avg, C

    # Final fit & test accuracy
    clf_final = SVC(kernel="precomputed", C=best_C).fit(K_full, ytr)
    test_acc = accuracy_score(yte, clf_final.predict(K_test))
    return test_acc, best_C


In [6]:
C_grid = [0.1, 0.5, 1, 2, 5, 10]

acc_simple, bestC_simple = tune_qsvm_precomputed(K_full_simple, K_test_simple, ytr, yte, C_grid)
acc_deep, bestC_deep     = tune_qsvm_precomputed(K_full_deep,   K_test_deep,   ytr, yte, C_grid)

print(f"Simple fmap: Best C={bestC_simple}, Test acc={acc_simple*100:.2f}%")
print(f"Deep   fmap: Best C={bestC_deep}, Test acc={acc_deep*100:.2f}%")


Simple fmap: Best C=0.1, Test acc=86.67%
Deep   fmap: Best C=5, Test acc=98.33%
